# Experiment 092: Non-Linear Mixture Features

**Rationale**: The best-work-here kernel uses non-linear mixture features that capture solvent-solvent interactions:
```python
# Current linear mixing:
mixture_feats = A * (1 - r) + B * r

# Non-linear mixing (from best-work-here):
mixture_feats = A * (1 - r) + B * r + 0.05 * A * B * r * (1 - r)
```

**Why this could change the CV-LB relationship**:
1. The non-linear term `0.05 * A * B * r * (1 - r)` captures interaction effects
2. This is fundamentally different from all 96 experiments we've run
3. Could reduce the intercept by better modeling mixture behavior

**Implementation**: Apply to GP+MLP+LGBM ensemble (best LB=0.0877)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer with NON-LINEAR MIXTURE FEATURES
class NonLinearMixtureFeaturizer:
    """Featurizer with non-linear mixture features from best-work-here kernel."""
    
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            
            r = X["SolventB%"].values.reshape(-1, 1) / 100.0
            
            if flip:
                r = 1 - r
                A_spange, B_spange = B_spange, A_spange
                A_drfp, B_drfp = B_drfp, A_drfp
                A_acs, B_acs = B_acs, A_acs
            
            # NON-LINEAR MIXTURE FEATURES (from best-work-here kernel)
            # Linear: A * (1 - r) + B * r
            # Non-linear: + 0.05 * A * B * r * (1 - r)
            linear_spange = A_spange * (1 - r) + B_spange * r
            interaction_spange = 0.05 * A_spange * B_spange * r * (1 - r)
            X_spange = linear_spange + interaction_spange
            
            linear_drfp = A_drfp * (1 - r) + B_drfp * r
            interaction_drfp = 0.05 * A_drfp * B_drfp * r * (1 - r)
            X_drfp = linear_drfp + interaction_drfp
            
            linear_acs = A_acs * (1 - r) + B_acs * r
            interaction_acs = 0.05 * A_acs * B_acs * r * (1 - r)
            X_acs = linear_acs + interaction_acs
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'NonLinearMixtureFeaturizer defined')
print(f'Feature dimension: {NonLinearMixtureFeaturizer().feats_dim}')

NonLinearMixtureFeaturizer defined
Feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            r = X["SolventB%"].values.reshape(-1, 1) / 100.0
            if flip:
                r = 1 - r
                A_spange, B_spange = B_spange, A_spange
            
            # Non-linear mixture for GP too
            linear = A_spange * (1 - r) + B_spange * r
            interaction_term = 0.05 * A_spange * B_spange * r * (1 - r)
            X_spange = linear + interaction_term
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# MLP Model
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64], output_dim=3):
        super().__init__()
        layers = []
        dim = input_dim
        for h in hidden_dims:
            layers += [nn.Linear(dim, h), nn.ReLU(), nn.Dropout(0.2)]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

print('SimpleMLP defined')

SimpleMLP defined


In [7]:
# GP+MLP+LGBM Ensemble with Non-Linear Mixture Features
class GPMLPLGBMEnsemble:
    def __init__(self, data='single'):
        self.mixed = (data == 'full')
        self.full_featurizer = NonLinearMixtureFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        self.gp_models = []
        self.mlp_model = None
        self.lgb_models = []
        
        # Ensemble weights (from exp_030)
        self.gp_weight = 0.3
        self.mlp_weight = 0.4
        self.lgb_weight = 0.3
        
    def train_model(self, train_X, train_Y):
        # Get features
        X_full = self.full_featurizer.featurize(train_X)
        X_simple = self.simple_featurizer.featurize(train_X)
        Y = train_Y.values
        
        # Scale features
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        # Train GP models (on simple features)
        self.gp_models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        for i in range(3):
            gp = GaussianProcessRegressor(
                kernel=kernel,
                n_restarts_optimizer=2,
                random_state=42,
                normalize_y=True
            )
            gp.fit(X_simple_scaled, Y[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y, dtype=torch.double).to(device)
        
        self.mlp_model = SimpleMLP(X_full_scaled.shape[1], [128, 64], 3).double().to(device)
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=1e-3)
        loss_fn = nn.MSELoss()
        
        self.mlp_model.train()
        for epoch in range(200):
            optimizer.zero_grad()
            pred = self.mlp_model(X_tensor)
            loss = loss_fn(pred, Y_tensor)
            loss.backward()
            optimizer.step()
        self.mlp_model.eval()
        
        # Train LGBM models
        self.lgb_models = []
        for i in range(3):
            model = lgb.LGBMRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                num_leaves=31,
                random_state=42,
                verbose=-1
            )
            model.fit(X_full_scaled, Y[:, i])
            self.lgb_models.append(model)
    
    def predict(self, test_X):
        # Get features
        X_full = self.full_featurizer.featurize(test_X)
        X_simple = self.simple_featurizer.featurize(test_X)
        
        # Scale features
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.zeros((len(test_X), 3))
        for i in range(3):
            gp_preds[:, i] = self.gp_models[i].predict(X_simple_scaled)
        
        # MLP predictions
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        with torch.no_grad():
            mlp_preds = self.mlp_model(X_tensor).cpu().numpy()
        
        # LGBM predictions
        lgb_preds = np.zeros((len(test_X), 3))
        for i in range(3):
            lgb_preds[:, i] = self.lgb_models[i].predict(X_full_scaled)
        
        # Ensemble
        ensemble_preds = (
            self.gp_weight * gp_preds +
            self.mlp_weight * mlp_preds +
            self.lgb_weight * lgb_preds
        )
        
        # Clip to valid range
        ensemble_preds = np.clip(ensemble_preds, 0, 1)
        
        return torch.tensor(ensemble_preds, dtype=torch.double)

print('GPMLPLGBMEnsemble with Non-Linear Mixture Features defined')

GPMLPLGBMEnsemble with Non-Linear Mixture Features defined


In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GPMLPLGBMEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:17, 17.40s/it]

2it [00:30, 14.88s/it]

3it [00:40, 12.51s/it]

4it [00:50, 11.48s/it]

...

21it [05:37, 19.30s/it]

22it [05:55, 18.78s/it]

23it [06:16, 19.55s/it]

24it [06:31, 18.16s/it]

24it [06:31, 16.31s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GPMLPLGBMEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:37, 37.34s/it]

2it [01:18, 39.37s/it]

3it [02:01, 41.26s/it]

4it [02:39, 40.09s/it]

...

10it [06:39, 39.53s/it]

11it [07:20, 40.02s/it]

12it [08:00, 39.77s/it]

13it [08:39, 39.66s/it]

13it [08:39, 39.97s/it]

In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [11]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest previous CV: 0.008092 (CatBoost+XGBoost)')
print(f'Best previous LB: 0.0877 (GP+MLP+LGBM)')
print(f'exp_030 baseline (GP+MLP+LGBM linear): CV 0.008298')
print(f'\nThis (GP+MLP+LGBM with Non-Linear Mixture): CV {overall_mse:.6f}')

if overall_mse < 0.008298:
    improvement = (0.008298 - overall_mse) / 0.008298 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_030!')
else:
    degradation = (overall_mse - 0.008298) / 0.008298 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_030')


=== CV SCORE VERIFICATION ===
Single Solvent MSE: 0.010909 (n=656)
Full Data MSE: 0.204018 (n=1227)
Overall MSE: 0.136743

Best previous CV: 0.008092 (CatBoost+XGBoost)
Best previous LB: 0.0877 (GP+MLP+LGBM)
exp_030 baseline (GP+MLP+LGBM linear): CV 0.008298

This (GP+MLP+LGBM with Non-Linear Mixture): CV 0.136743

✗ WORSE: 1547.90% worse than exp_030


In [ ]:
# Debug: Check what's happening with the full data predictions
print("Checking predictions...")
print(f"Single solvent predictions range: [{preds_single.min():.4f}, {preds_single.max():.4f}]")
print(f"Full data predictions range: [{preds_full.min():.4f}, {preds_full.max():.4f}]")
print(f"Single solvent actuals range: [{actuals_single.min():.4f}, {actuals_single.max():.4f}]")
print(f"Full data actuals range: [{actuals_full.min():.4f}, {actuals_full.max():.4f}]")

# Check for NaN or inf
print(f"\nNaN in single preds: {np.isnan(preds_single).sum()}")
print(f"NaN in full preds: {np.isnan(preds_full).sum()}")
print(f"Inf in single preds: {np.isinf(preds_single).sum()}")
print(f"Inf in full preds: {np.isinf(preds_full).sum()}")